# 소켓 프로그래밍 (Socket Programming)

## 1. 소켓 (Socket)
- 사전적으로 "구멍", "연결", "콘센트" 등의 의미를 가집니다.
- 주로 전기 부품을 규격에 따라 연결할 수 있게 만들어진 "구멍 형태의 연결부"를 일컫는 단어로 가정에서 흔히 볼 수 있는 콘센트 구멍을 떠올리면 쉽게 이해가 가능합니다.
- 전기를 필요로하는 디바이스 또는 전기를 공급 받을 수 있도록, 전기 공급 인프라 환경에 연결할 수 있게 만들어진 연결부가 "소켓(socket)"인 것
- 네트워크 프로그래밍에서의 소켓(socket)에 대한 의미도 사전적 의미를 크게 벗어나지 않습니다. 프로그램이 네트워크에 데이터를 송수신할 수 있도록, "네트워크 환경에 연결할 수 있게 만들어진 연결부"가 바로 "네트워크 소켓(Socket)"입니다.
- 정식명칭은 "TCP소켓" 또는 "TCP/IP 소켓"이라고 부릅니다. (UDP에서 동작하는 소켓은 "UDP 소켓"이라고 합니다.)

## 2. TCP/IP 소켓 프로그래밍 (Socket Programming)
> 소켓을 사용하여 네트워크 통신 기능을 구현하는 과정, 즉 소켓 프로그래밍은 개념만큼 단순하지 않습니다.
- `소켓을 만드는 것과 만들어진 소켓을 통해 데이터를 주고 받는 절차`에 대한 이해가 필요
- 운영체제 및 프로그래밍 언어에 종속적으로 제공되는 `소켓 API 사용법`을 숙지해야 하기 때문
- 그외에도 케이블 분리로 인한 네트워크 단절, 트래픽 증가에 따른 데이터 전송 지연, 시스템 리소스 관리 문제로 인한 에러 등 네트워크 환경에서 발생할 수 있는 다양한 예외사항에 대해서도 처리가 필요하기 때문

### 2-1. 클라이언트 소켓(Client Socket)과 서버 소켓(Server Socket)
> 두 개의 시스템(또는 프로세스)이 소켓을 통해 네트워크 연결(Connection)을 만들기 위해서는, 최초 어느 한 곳에서 그 대상이 되는 곳으로 연결을 요청해야 합니다. IP주소와 번호로 식별되는 대상에게, 자신이 데이터 송수신을 위한 네트워크 연결을 수립할 의사가 있음을 알리는 것

> Q. 최초 한 곳에서 무작정 연결을 시도한다고 해서, 그 요청이 무조건 받아들여지고 연결이 만들어져 데이터를 주고 받을 수 있게 될까?
- 정답은 아닙니다. 한 곳에서 연결 요청을 보낸다고 하더라도 그 대상 시스템이 그 요청을 받아들일 준비가 되어 있지 않다면, 해당 요청은 무시되고 연결은 만들어지지 않습니다.\
그러므로 요청을 받아들이는 곳에서는 어떤 연결 요청(일반적으로 포트로 번호 식별)을 받아들일 것인지를 미리 시스템에 등록하여, 요청이 수신되었을 때 해당 요청을 처리할 수 있도록 준비해야 합니다.

> 두 개의 시스템(또는 프로세스)이 소켓을 통해 데이터 통신을 위한 연결(Connection)을 만들기 위해서는, 연결 요청을 보내는지 또는 요청을 받아들이는지에 따라 소켓의 역할이 나뉘게 되는데, 전자에 사용되는 소켓을 클라이언트 소켓(Client Socket), 후자에 사용되는 소켓을 서버 소켓(Server Socket)이라고 합니다.
- 주의 : 앞서 설명한 내용을 보면 마치 클라이언트 소켓과 서버 소켓이 태생적으로 구조가 다른, 전혀 별개의 소켓인 것처럼 여겨질 수 있다는 것.\
하지만 두 소켓은 동일합니다. 소켓의 역할과 구현 절차 구분을 위해 다르게 부르는 것일 뿐, 전혀 다른 형태의 소켓이 아니라는 것.\
즉, 호출되는 API함수의 종류와 순서들이 다를 뿐입니다.

### 2-2. 소켓 API(Socket API)실행 흐름.
> 각 소켓이 처리되는 흐름을 간단한 문장으로 정리
- 클라이언트 소켓 : 처음 소켓을 [1]생성(create)한 다음, 서버측에 [2]연결(connect)을 요청합니다. 그리고 서버 소켓에서 연결이 받아들여지면 데이터를 [3]송수신(send/recv)하고, 모든 처리가 완료되면 소켓을 [4]닫습니다.(close).
- 서버 소켓 : 첫 단계 [1]생성(create)하는것, 서버가 사용할 IP주소와 포트 번호를 생성한 소켓에 [2]결합(bind)하는 것입니다. 그다음 클라이언트로부터 연결 요청이 수신되는지 [3]주시(listen)하고, 요청이 수신되면 요청을 [4]받아들여(accept) 데이터 통신을 위한 소켓을 생성합니다. 일단 새로운 소켓을 통해 연결이 수립(ESTABLISHED)되면, 클라이언트와 마찬가지로 데이터를 [5]송수신(send/recv)할 수 있습니다. 마지막으로 데이터 송수신이 완료되면, 소켓(socket)을 [6]닫습니다.(close)